In [129]:
import pywt
import os
from pylab import *
from numpy import*
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score

#### Обработка графиков ЭКГ и преобразование в таблицу

In [2]:
def get_ecg(key):
  way = 'train/' + key + '.npy'
  ecgsig = np.load(way).transpose().reshape((60000,))
  x = np.linspace(0,  1,  num = 2048)
  data = np.sin(250 * np.pi * x**2)
  coefs = pywt.downcoef('a', data, 'db20', mode='symmetric', level=1)
  st='sym5'
  coeffs = pywt.wavedec(ecgsig, st, level=5)
  row = coeffs[0]
  return row

In [3]:
def into_one_table(keys):
  row = get_ecg(keys[0])
  row = np.insert(row, 0, int(keys[0].strip('_hr')))
  table = [row]
  for i in range(target.index.start + 1, target.index.stop):
    row = get_ecg(keys[i])
    row = np.insert(row, 0, int(keys[i].strip('_hr')))
    table = np.vstack([table, row])
  table = pd.DataFrame(table)
  return table

In [4]:
target = pd.Series(list(map(lambda x: x[:-4], os.listdir('train')[:-2])))
ECG_data = into_one_table(target)

In [5]:
ECG_data[0] = ECG_data[0].astype('int')
ECG_data = ECG_data.rename(columns={0: 'ecg_id'})

In [6]:
ECG_data = ECG_data.sort_values(by=['ecg_id'], ascending=True)

In [7]:
ECG_data

,ecg_id,1,2,3,4,5,6,7,8,9,...,1874,1875,1876,1877,1878,1879,1880,1881,1882,1883
0,9,-0.170442,-0.066933,-0.140079,-0.478502,-1.245167,-0.979738,-0.738943,-0.476043,-0.349288,...,0.558936,0.551860,0.551007,0.561618,0.556675,0.645774,0.744957,0.649421,0.667106,0.731759
1,34,0.399648,0.579593,0.460080,-0.077166,-1.270282,-1.040847,-1.689153,-2.481032,-2.205554,...,1.113606,1.088111,1.099680,1.132341,1.130250,1.788834,2.455783,1.818630,1.938019,2.363243
2,43,0.411589,0.365335,0.399006,0.554728,0.911140,0.774487,0.576258,0.404740,0.229235,...,0.341701,0.340029,0.342162,0.341734,0.340499,0.346990,0.366654,0.343959,0.348800,0.365320
3,52,-0.162251,-0.159025,-0.159327,-0.163111,-0.158814,-0.159398,-0.162262,-0.174764,-0.179583,...,-0.267047,-0.258175,-0.261318,-0.274144,-0.273755,-0.519687,-0.765086,-0.532849,-0.575938,-0.730437
4,57,-0.268330,-0.279812,-0.270770,-0.232069,-0.136626,-0.155710,-0.117283,-0.096148,-0.074360,...,-0.304742,-0.297821,-0.302716,-0.310384,-0.311176,-0.513279,-0.714933,-0.521431,-0.558040,-0.686983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,21784,-0.094946,-0.097736,-0.095932,-0.087848,-0.070772,-0.076037,-0.082161,-0.089443,-0.092388,...,0.005842,0.004748,0.005003,0.006885,0.006829,0.041791,0.079657,0.044349,0.050701,0.074361
2097,21795,-0.194416,-0.182553,-0.191956,-0.238293,-0.352612,-0.314071,-0.319038,-0.313175,-0.285890,...,0.120157,0.117738,0.121368,0.121892,0.122540,0.189928,0.260724,0.190226,0.204078,0.251573
2098,21825,-0.004072,-0.000198,-0.003268,-0.017719,-0.054790,-0.048271,-0.088630,-0.084493,-0.070101,...,-0.128999,-0.124998,-0.127924,-0.131497,-0.131004,-0.222249,-0.318699,-0.224482,-0.242503,-0.305902
2099,21831,-0.128454,-0.127641,-0.128944,-0.136319,-0.158699,-0.147968,-0.147763,-0.142817,-0.138197,...,-0.159457,-0.155589,-0.158779,-0.161389,-0.160510,-0.233738,-0.313892,-0.233884,-0.249503,-0.303710


## N-граммы

In [8]:
import neurokit2
import sleepecg
import wfdb.processing

In [9]:
def find_peaks(ecgsignal, peak):
    return wfdb.processing.correct_peaks(ecgsignal, rpeaks,
            search_radius=36, smooth_window_size=50, peak_dir=peak)

In [10]:
def get_cycle(ecgsignal, rpeaks): # разбиение ЭКГ на циклы
  ecgsignal_split = []
  cycle = []
  rpeaks = rpeaks[::-1]
  f = True #флаг конца массива с пиками
  f1 = True #флаг вхождения первого пика
  for i in range(len(ecgsignal)):
    if f and i == rpeaks[-1]:
      if f1:
        f1 = False
        cycle = []
      else:
        ecgsignal_split.append(cycle)
        cycle = []
      rpeaks = rpeaks[:-1]
      if f and len(rpeaks) == 0:
        f = False
    cycle.append(ecgsignal[i])
  return ecgsignal_split

In [11]:
def sign(x):
   return 1 if x > 0 else -1

In [12]:
def RTL(ecgsignal_split):
    sp = [[(i[0] - min(i[:10])), len(i), ((i[0] - min(i[:10]))/len(i))] for i in ecgsignal_split]
    s = []
    for i in range(len(sp) - 1):
        R = sp[i + 1][0] - sp[i][0]
        T = sp[i + 1][1] - sp[i][1]
        L = sp[i + 1][2] - sp[i][2]
        s.append([R, T, L])
    return s

In [13]:
def letters(data): # кодирование цикла по В.М. Успенскому 
  if data[0] == 1:
    if data[1] == -1:
      return 'C'
    else:
      if data[2] == -1:
        return 'E'
      return 'A'
  else:
    if data[1] == 1:
      return 'D'
    else:
      if data[2] == 1:
        return 'B'
      return 'F'

In [14]:
def encoding(ecgsignal_split):
    s = [[sign(i) for i in k] for k in RTL(ecgsignal_split)]
    stroka = ''
    for i in s:
        stroka += letters(i)
    return stroka

In [15]:
ECG_data_array = np.array(ECG_data)

In [16]:
ecg_ids = []
encoding_ecgs = []

In [17]:
for ecgsignal in ECG_data_array:
    ecg_id = ecgsignal[0]
    ecgsignal = ecgsignal[1:]
    
    _, results = neurokit2.ecg_peaks(ecgsignal, sampling_rate=100)
    rpeaks = results["ECG_R_Peaks"]
    rpeaks = sleepecg.detect_heartbeats(ecgsignal, fs=100)
    rpeaks = wfdb.processing.xqrs_detect(ecgsignal, fs=100, verbose=False)
    
    rpeaks = find_peaks(ecgsignal, 'up')
    
    ecgsignal_split = get_cycle(ecgsignal, rpeaks)
    encoding_ecg = encoding(ecgsignal_split)
    
    ecg_ids.append(ecg_id)
    encoding_ecgs.append(encoding_ecg)

In [18]:
encoding_ecg = pd.Series(encoding_ecgs, index=ecg_ids)

In [19]:
encoding_ecg

9.0                   DDCADDCC
34.0       DDDCCCFFFDDFCCCACFE
43.0               BFABCFADDEC
52.0               DDEFBCCBCCF
57.0                 DDCADBCDD
                  ...         
21784.0         DDCCFFAFAFFAFC
21795.0                       
21825.0              ECCDDDECB
21831.0           DAFCCDDDCFCD
21834.0              CDFCCDAFF
Length: 2101, dtype: object

In [20]:
gts = pd.read_csv('train/train_gts.csv')
gts.record_name = gts.record_name.map(lambda x: float(x[:-3]))
gts = gts.sort_values(by='record_name')
gts.insert(2, 'encoding_ecg', array(encoding_ecg))
gts

,record_name,myocard,encoding_ecg
265,9.0,0,DDCADDCC
1407,34.0,0,DDDCCCFFFDDFCCCACFE
1833,43.0,0,BFABCFADDEC
441,52.0,0,DDEFBCCBCCF
1834,57.0,0,DDCADBCDD
...,...,...,...
1753,21784.0,0,DDCCFFAFAFFAFC
2008,21795.0,0,
72,21825.0,0,ECCDDDECB
2100,21831.0,0,DAFCCDDDCFCD


In [21]:
sick = gts[gts.myocard == 1].encoding_ecg
healthy = gts[gts.myocard == 0].encoding_ecg

In [23]:
from nltk.util import ngrams

In [99]:
sp_sick= []

for i in sick:
    n = list(map(lambda x: ''.join(list(x)), list(ngrams(list(i), 5))))
    for k in n:
        sp_sick.append(k)

In [100]:
sp_healthy= []

for i in healthy:
    n = list(map(lambda x: ''.join(list(x)), list(ngrams(list(i), 5))))
    for k in n:
        sp_healthy.append(k)
    

In [102]:
def counter_and_sort(arr):
  quantity = {}
  for i in arr:
    if i in quantity:
      quantity[i] += 1
    else:
      quantity[i] = 1
  quantity = pd.Series(list(quantity.values()), index=list(quantity.keys())).sort_values(ascending=False)
  return quantity



In [103]:
q_sick = counter_and_sort(sp_sick)
q_health = counter_and_sort(sp_healthy)

In [107]:
only_sick = list(set(q_sick.index) - set(q_health.index))
only_health = list(set(q_health.index) - set(q_sick.index))

In [131]:
popular_sick = q_sick.iloc[:len(q_sick)//2]
popular_health = q_health.iloc[:len(q_health)//2]

In [135]:
popular_sick_only = list(set(popular_sick.index) - set(popular_health.index))
popular_health_only = list(set(popular_health.index) - set(popular_sick.index))

In [110]:
array(gts)

array([[9.0, 0, 'DDCADDCC'],
       [34.0, 0, 'DDDCCCFFFDDFCCCACFE'],
       [43.0, 0, 'BFABCFADDEC'],
       ...,
       [21825.0, 0, 'ECCDDDECB'],
       [21831.0, 0, 'DAFCCDDDCFCD'],
       [21834.0, 1, 'CDFCCDAFF']], dtype=object)

In [120]:
ngramms_ecgs = []
for ecg in array(gts):
    ngramms_ecg = set()
    n = list(map(lambda x: ''.join(list(x)), list(ngrams(list(ecg[2]), 5))))
    for k in n:
        ngramms_ecg.add(k)
    ngramms_ecgs.append(list(ngramms_ecg))

In [382]:
ngramms_ecgs

[['ADDCC', 'DDCAD', 'CADDC', 'DCADD'],
 ['FDDFC',
  'CCCAC',
  'CFFFD',
  'FFDDF',
  'FFFDD',
  'CACFE',
  'CCACF',
  'CCCFF',
  'DDFCC',
  'DCCCF',
  'DDCCC',
  'DFCCC',
  'FCCCA',
  'DDDCC',
  'CCFFF'],
 ['FADDE', 'ADDEC', 'BCFAD', 'ABCFA', 'BFABC', 'CFADD', 'FABCF'],
 ['CBCCF', 'FBCCB', 'BCCBC', 'DDEFB', 'DEFBC', 'CCBCC', 'EFBCC'],
 ['DCADB', 'ADBCD', 'CADBC', 'DDCAD', 'DBCDD'],
 ['BBCCD', 'DBBCC', 'CCDDA', 'BCCDD', 'DDBBC'],
 ['BADDC', 'AFCBA', 'CBADD', 'FCBAD', 'CAFCB'],
 ['FDCFD', 'CCFDC', 'FFCCF', 'CFDCF', 'DCFDA', 'FCCFD'],
 ['CCFFC',
  'FCCCD',
  'DDECF',
  'CFFCC',
  'CCCDC',
  'FFCCF',
  'CCDDD',
  'FCCFF',
  'FFCCC',
  'CDDDE',
  'DDDEC',
  'DECFF',
  'ECFFC'],
 ['BCEDB', 'EDBDE', 'CEDBD', 'EBCED', 'BEBCE', 'DBDEC'],
 ['DFCDE', 'DDCED', 'EDFCD', 'BDDCE', 'DCEDF', 'CEDFC'],
 ['CDDCD', 'CCDDC', 'DCCDD', 'DDCDD', 'DCDDC', 'CDDCF'],
 ['DCDAA',
  'DABDC',
  'DCDAB',
  'CDABD',
  'ABDCD',
  'AADDB',
  'DAADD',
  'BDCDA',
  'CDAAD'],
 ['CFDDE', 'CCFDD', 'DDCFC', 'FDDEE', 'DCFCC', 

In [384]:
def func(ng):
    ng_health = sum(list(map(lambda x: 1 if x in only_health else 0, ng)))
    ng_sick = sum(list(map(lambda x: 1 if x in only_sick else 0, ng)))
    
    ng_popular_health = sum(list(map(lambda x: 1 if x in popular_health_only else 0, ng)))
    ng_popular_sick = sum(list(map(lambda x: 1 if x in popular_sick_only else 0, ng)))
    
    return [ng_health, ng_sick, ng_popular_health, ng_popular_sick]

In [400]:
arr = pd.DataFrame([func(ng) for ng in ngramms_ecgs])

In [401]:
arr

,0,1,2,3
0,0.0,0.0,1.0,0.0
1,8.0,0.0,10.0,0.0
2,6.0,0.0,4.0,0.0
3,7.0,0.0,3.0,0.0
4,3.0,0.0,4.0,0.0
...,...,...,...,...
2096,2.0,0.0,5.0,0.0
2097,0.0,0.0,0.0,0.0
2098,4.0,0.0,4.0,0.0
2099,4.0,0.0,4.0,0.0


In [399]:
from sklearn.model_selection import train_test_split

In [402]:
X_train, X_test, y_train, y_test = train_test_split(arr, gts.myocard, random_state=0)

In [408]:
from sklearn.linear_model import LogisticRegression

In [409]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [410]:
accuracy_score(y_train, model.predict(X_train))

0.8971428571428571

In [411]:
precision_score(y_train, model.predict(X_train))

0.9556962025316456

In [412]:
recall_score(y_train, model.predict(X_train))

0.4934640522875817

In [414]:
from catboost import CatBoostClassifier

In [424]:
cat = CatBoostClassifier()
cat.fit(X_train, y_train)

Learning rate set to 0.012508
0:	learn: 0.6706694	total: 17.8ms	remaining: 17.8s
1:	learn: 0.6436612	total: 21.1ms	remaining: 10.5s
2:	learn: 0.6308520	total: 24.6ms	remaining: 8.17s
3:	learn: 0.6125041	total: 48.7ms	remaining: 12.1s
4:	learn: 0.5923970	total: 53.1ms	remaining: 10.6s
5:	learn: 0.5704670	total: 59.5ms	remaining: 9.86s
6:	learn: 0.5529860	total: 71.1ms	remaining: 10.1s
7:	learn: 0.5326285	total: 76.1ms	remaining: 9.44s
8:	learn: 0.5168678	total: 90.1ms	remaining: 9.92s
9:	learn: 0.5001869	total: 95.8ms	remaining: 9.49s
10:	learn: 0.4843971	total: 103ms	remaining: 9.25s
11:	learn: 0.4709328	total: 110ms	remaining: 9.03s
12:	learn: 0.4654942	total: 114ms	remaining: 8.64s
13:	learn: 0.4539401	total: 132ms	remaining: 9.27s
14:	learn: 0.4479111	total: 136ms	remaining: 8.93s
15:	learn: 0.4343202	total: 140ms	remaining: 8.63s
16:	learn: 0.4245978	total: 160ms	remaining: 9.27s
17:	learn: 0.4143326	total: 168ms	remaining: 9.17s
18:	learn: 0.4091595	total: 173ms	remaining: 8.93s
1

160:	learn: 0.2144210	total: 1.23s	remaining: 6.42s
161:	learn: 0.2143622	total: 1.24s	remaining: 6.42s
162:	learn: 0.2142458	total: 1.26s	remaining: 6.46s
163:	learn: 0.2140911	total: 1.26s	remaining: 6.43s
164:	learn: 0.2139532	total: 1.28s	remaining: 6.48s
165:	learn: 0.2138108	total: 1.29s	remaining: 6.46s
166:	learn: 0.2138024	total: 1.29s	remaining: 6.43s
167:	learn: 0.2136393	total: 1.3s	remaining: 6.44s
168:	learn: 0.2135113	total: 1.32s	remaining: 6.49s
169:	learn: 0.2134804	total: 1.32s	remaining: 6.47s
170:	learn: 0.2134187	total: 1.33s	remaining: 6.44s
171:	learn: 0.2132682	total: 1.34s	remaining: 6.44s
172:	learn: 0.2132134	total: 1.34s	remaining: 6.42s
173:	learn: 0.2131599	total: 1.35s	remaining: 6.4s
174:	learn: 0.2130226	total: 1.36s	remaining: 6.41s
175:	learn: 0.2129213	total: 1.36s	remaining: 6.38s
176:	learn: 0.2128056	total: 1.37s	remaining: 6.36s
177:	learn: 0.2127165	total: 1.39s	remaining: 6.4s
178:	learn: 0.2126070	total: 1.4s	remaining: 6.43s
179:	learn: 0.21

328:	learn: 0.2075652	total: 2.48s	remaining: 5.06s
329:	learn: 0.2075614	total: 2.49s	remaining: 5.05s
330:	learn: 0.2075531	total: 2.5s	remaining: 5.05s
331:	learn: 0.2075263	total: 2.51s	remaining: 5.04s
332:	learn: 0.2075051	total: 2.51s	remaining: 5.03s
333:	learn: 0.2074666	total: 2.52s	remaining: 5.02s
334:	learn: 0.2074632	total: 2.52s	remaining: 5.01s
335:	learn: 0.2074489	total: 2.53s	remaining: 5s
336:	learn: 0.2074178	total: 2.54s	remaining: 4.99s
337:	learn: 0.2074165	total: 2.54s	remaining: 4.97s
338:	learn: 0.2074111	total: 2.54s	remaining: 4.96s
339:	learn: 0.2074091	total: 2.55s	remaining: 4.95s
340:	learn: 0.2073889	total: 2.56s	remaining: 4.95s
341:	learn: 0.2073858	total: 2.56s	remaining: 4.93s
342:	learn: 0.2073792	total: 2.57s	remaining: 4.92s
343:	learn: 0.2073759	total: 2.57s	remaining: 4.91s
344:	learn: 0.2073675	total: 2.58s	remaining: 4.9s
345:	learn: 0.2073356	total: 2.59s	remaining: 4.89s
346:	learn: 0.2073152	total: 2.59s	remaining: 4.88s
347:	learn: 0.207

515:	learn: 0.2059470	total: 3.71s	remaining: 3.48s
516:	learn: 0.2059331	total: 3.71s	remaining: 3.47s
517:	learn: 0.2059324	total: 3.72s	remaining: 3.46s
518:	learn: 0.2059213	total: 3.72s	remaining: 3.45s
519:	learn: 0.2059170	total: 3.73s	remaining: 3.44s
520:	learn: 0.2059122	total: 3.73s	remaining: 3.43s
521:	learn: 0.2059074	total: 3.75s	remaining: 3.44s
522:	learn: 0.2059056	total: 3.76s	remaining: 3.43s
523:	learn: 0.2059049	total: 3.76s	remaining: 3.42s
524:	learn: 0.2058958	total: 3.77s	remaining: 3.41s
525:	learn: 0.2058962	total: 3.77s	remaining: 3.4s
526:	learn: 0.2058926	total: 3.78s	remaining: 3.39s
527:	learn: 0.2058888	total: 3.79s	remaining: 3.38s
528:	learn: 0.2058879	total: 3.79s	remaining: 3.37s
529:	learn: 0.2058774	total: 3.79s	remaining: 3.37s
530:	learn: 0.2058779	total: 3.8s	remaining: 3.36s
531:	learn: 0.2058757	total: 3.81s	remaining: 3.35s
532:	learn: 0.2058680	total: 3.82s	remaining: 3.35s
533:	learn: 0.2058677	total: 3.83s	remaining: 3.34s
534:	learn: 0.

686:	learn: 0.2052020	total: 4.98s	remaining: 2.27s
687:	learn: 0.2051998	total: 4.99s	remaining: 2.26s
688:	learn: 0.2051916	total: 5.01s	remaining: 2.26s
689:	learn: 0.2051900	total: 5.01s	remaining: 2.25s
690:	learn: 0.2051828	total: 5.02s	remaining: 2.24s
691:	learn: 0.2051812	total: 5.02s	remaining: 2.23s
692:	learn: 0.2051752	total: 5.03s	remaining: 2.23s
693:	learn: 0.2051708	total: 5.05s	remaining: 2.23s
694:	learn: 0.2051644	total: 5.06s	remaining: 2.22s
695:	learn: 0.2051595	total: 5.06s	remaining: 2.21s
696:	learn: 0.2051546	total: 5.07s	remaining: 2.2s
697:	learn: 0.2051493	total: 5.07s	remaining: 2.19s
698:	learn: 0.2051445	total: 5.08s	remaining: 2.19s
699:	learn: 0.2051440	total: 5.08s	remaining: 2.18s
700:	learn: 0.2051424	total: 5.09s	remaining: 2.17s
701:	learn: 0.2051392	total: 5.09s	remaining: 2.16s
702:	learn: 0.2051391	total: 5.1s	remaining: 2.15s
703:	learn: 0.2051314	total: 5.11s	remaining: 2.15s
704:	learn: 0.2051231	total: 5.13s	remaining: 2.15s
705:	learn: 0.

860:	learn: 0.2045352	total: 6.63s	remaining: 1.07s
861:	learn: 0.2045315	total: 6.64s	remaining: 1.06s
862:	learn: 0.2045285	total: 6.65s	remaining: 1.06s
863:	learn: 0.2045254	total: 6.67s	remaining: 1.05s
864:	learn: 0.2045233	total: 6.67s	remaining: 1.04s
865:	learn: 0.2045194	total: 6.69s	remaining: 1.03s
866:	learn: 0.2045175	total: 6.7s	remaining: 1.03s
867:	learn: 0.2045141	total: 6.72s	remaining: 1.02s
868:	learn: 0.2045120	total: 6.74s	remaining: 1.01s
869:	learn: 0.2045081	total: 6.76s	remaining: 1.01s
870:	learn: 0.2045058	total: 6.76s	remaining: 1s
871:	learn: 0.2045018	total: 6.77s	remaining: 994ms
872:	learn: 0.2044996	total: 6.78s	remaining: 986ms
873:	learn: 0.2044957	total: 6.79s	remaining: 978ms
874:	learn: 0.2044933	total: 6.79s	remaining: 971ms
875:	learn: 0.2044894	total: 6.8s	remaining: 963ms
876:	learn: 0.2044874	total: 6.82s	remaining: 957ms
877:	learn: 0.2044849	total: 6.83s	remaining: 948ms
878:	learn: 0.2044814	total: 6.84s	remaining: 942ms
879:	learn: 0.204

In [425]:
accuracy_score(y_train, cat.predict(X_train))

0.8971428571428571

In [426]:
precision_score(y_train, cat.predict(X_train))

0.9864864864864865

In [427]:
recall_score(y_train, cat.predict(X_train))

0.477124183006536